In [1]:
#had a lot of issues trying to install the "imputeLCMD" package, so this was how I did it
# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install(c("impute", "pcaMethods"))

# install.packages("imputeLCMD")

In [2]:
setwd("/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.1. Data Imputation/Input")
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.1. Data Imputation/Output')
cur_date = "020823"

library(missForest)
library(imputeLCMD)
library(readxl)
library(tidyverse)
library(tmvtnorm)

# reading in files
ozone_data = data.frame(read_excel("Ozone_Sterol_Asthma_data_042622.xlsx", sheet = 2)) 

Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix

Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
and has been superseded by the norm2 package.

Loading required package: pcaMethods

Loading required package: Biobase

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownam

Goal is to generate (impute) values for missing data. Given that this dataset contains various categories (ie. cell differentials, cytokines, lung function, sterol data) a different method will be employed for each.

> Lung Function Data: Random Forest (RF)

> Soluble Mediators (everything else): Quantile Regression Imputation of Left-Censored Data (QRILC)

Acutally no longer needed to impute lung function data, because there was no missing data. 

In [3]:
head(ozone_data)

,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,4,1,AS_1,AS,A,Post,NR,NR,Lung_Function,FEV1,4.3899999999999899
2,4,1,AS_1,AS,A,Pre,NR,NR,Lung_Function,FEV1,4.5999999999999899
3,7,2,AS_2,AS,A,Post,R,NR,Lung_Function,FEV1,2.96
4,7,2,AS_2,AS,A,Pre,R,NR,Lung_Function,FEV1,3.08
5,11,3,AS_3,AS,A,Post,NR,R,Lung_Function,FEV1,4.24
6,11,3,AS_3,AS,A,Pre,NR,R,Lung_Function,FEV1,4.5199999999999898


In [4]:
# total number of samples
dim(ozone_data)[1]

[1] 7248

In [5]:
# determing what subjects have missing data for all variables delinated based on the dfs the data will be split into
subject_presence_df = ozone_data %>%
    mutate(count = ifelse(Value != "NA", 1, 0)) %>%
    group_by(Category, Sample_Type, Time_Point, Subject_ID) %>%
    summarize(Subject_Presence_Percentage = sum(count)/ length(unique(Variable)) * 100) %>%
    arrange(Subject_Presence_Percentage)
head(subject_presence_df)

`summarise()` has grouped output by 'Category', 'Sample_Type', 'Time_Point'.
You can override using the `.groups` argument.


Category,Sample_Type,Time_Point,Subject_ID,Subject_Presence_Percentage
<chr>,<chr>,<chr>,<chr>,<dbl>
Cytokine,S,Pre,AS_3,0.00000
Sterol,S,Pre,AS_3,0.00000
Cytokine,P,Post,AS_1,96.77419
Cytokine,P,Post,AS_10,96.77419
Cytokine,P,Post,AS_11,96.77419
Cytokine,P,Post,AS_12,96.77419


Based on this, subject AS_3 will be removed from all cytokine and sterol samples pre exposure. 

In [6]:
# determining variables that have missing data for all variables delineated based on the dfs the data will be 
# split into
variable_presence_df = ozone_data %>%
    mutate(count = ifelse(Value != "NA", 1, 0)) %>%
    group_by(Sample_Type, Time_Point, Variable) %>%
    # calculating the percentage of variables with data in all subjects
    summarize(Variable_Presence_Percentage = sum(count)/ length(unique(Subject_ID)) * 100) %>%
    arrange(Variable_Presence_Percentage)
head(variable_presence_df)

`summarise()` has grouped output by 'Sample_Type', 'Time_Point'. You can
override using the `.groups` argument.


Sample_Type,Time_Point,Variable,Variable_Presence_Percentage
<chr>,<chr>,<chr>,<dbl>
P,Post,Utero,0.00000
P,Pre,Utero,0.00000
S,Post,MPO,87.50000
S,Pre,MPO,87.50000
S,Pre,24OH_Chol,95.83333
S,Pre,25OH_Chol,95.83333


Based on this, cytokine, Utero, will be removed from plasma samples pre and post exposure. 

In [7]:
# implementing background filter where a variable needs to be present in at least 25% of subjects
# and where subject needs to have at least 25% of data for variables within a category/ time point/ sample
variable1_to_remove_df = ozone_data %>%
    filter(Sample_Type == "P" & Variable == "Utero")
variable2_to_remove_df = ozone_data %>%
    filter(Sample_Type == "S", Time_Point == "Pre", Category == "Cytokine", Subject_ID == "AS_3") 
variable3_to_remove_df = ozone_data %>%
    filter(Sample_Type == "S", Time_Point == "Pre", Category == "Sterol", Subject_ID == "AS_3") 
variables_to_remove_df = rbind(variable1_to_remove_df, variable2_to_remove_df, variable3_to_remove_df)

head(variables_to_remove_df)

,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,4,1,AS_1,AS,P,Post,NR,NR,Cytokine,Utero,NA
2,4,1,AS_1,AS,P,Pre,NR,NR,Cytokine,Utero,NA
3,7,2,AS_2,AS,P,Post,R,NR,Cytokine,Utero,NA
4,7,2,AS_2,AS,P,Pre,R,NR,Cytokine,Utero,NA
5,11,3,AS_3,AS,P,Post,NR,R,Cytokine,Utero,NA
6,11,3,AS_3,AS,P,Pre,NR,R,Cytokine,Utero,NA


In [8]:
# number of samples being removed
dim(variables_to_remove_df)[1]

[1] 97

In [9]:
# number of samples to expect in the final df
dim(ozone_data)[1] - dim(variables_to_remove_df)[1]

[1] 7151

In [10]:
# only keeping rows in ozone data that aren't in the other dataframe
filtered_ozone_data = anti_join(ozone_data, variables_to_remove_df[1:11]) %>%
    #making value col numeric instead of character
    mutate(Value = ifelse(Value == "NA", NA, as.numeric(Value))) 

head(filtered_ozone_data)

Joining, by = c("Original_Subject_No", "Subject_No", "Subject_ID",
"Disease_Status", "Sample_Type", "Time_Point", "Inflammatory_Responder",
"Lung_Responder", "Category", "Variable", "Value")
Warning message in ifelse(Value == "NA", NA, as.numeric(Value)):
“NAs introduced by coercion”


,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,4,1,AS_1,AS,A,Post,NR,NR,Lung_Function,FEV1,4.39
2,4,1,AS_1,AS,A,Pre,NR,NR,Lung_Function,FEV1,4.60
3,7,2,AS_2,AS,A,Post,R,NR,Lung_Function,FEV1,2.96
4,7,2,AS_2,AS,A,Pre,R,NR,Lung_Function,FEV1,3.08
5,11,3,AS_3,AS,A,Post,NR,R,Lung_Function,FEV1,4.24
6,11,3,AS_3,AS,A,Pre,NR,R,Lung_Function,FEV1,4.52


In [11]:
# now splitting dataset based on category, sample, time point
preimputed_df = filtered_ozone_data %>%
    group_by(Category, Sample_Type, Time_Point) %>%
    group_split()

cell_differential_plasma_post_df = preimputed_df[[1]]
cell_differential_plasma_pre_df = preimputed_df[[2]]
cell_differential_sputum_post_df = preimputed_df[[3]]
cell_differential_sputum_pre_df = preimputed_df[[4]]
cytokine_plasma_post_df = preimputed_df[[5]]
cytokine_plasma_pre_df = preimputed_df[[6]]
cytokine_sputum_post_df = preimputed_df[[7]]
cytokine_sputum_pre_df = preimputed_df[[8]]
sterol_plasma_post_df = preimputed_df[[11]]
sterol_plasma_pre_df = preimputed_df[[12]]
sterol_sputum_post_df = preimputed_df[[13]]
sterol_sputum_pre_df = preimputed_df[[14]]

# no longer needed for imputation, but kept for when all imputed data is combined
lung_function_airways_post_df = preimputed_df[[9]] 
lung_function_airways_pre_df = preimputed_df[[10]] 

# QRILC Imputation

In [12]:
head(cell_differential_plasma_post_df)

Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Sample_Weight,817
7,2,AS_2,AS,P,Post,R,NR,Cell_Differential,Sample_Weight,7992
11,3,AS_3,AS,P,Post,NR,R,Cell_Differential,Sample_Weight,2791
13,4,AS_4,AS,P,Post,R,NR,Cell_Differential,Sample_Weight,3530
15,5,AS_5,AS,P,Post,R,NR,Cell_Differential,Sample_Weight,3830
16,6,AS_6,AS,P,Post,NR,NR,Cell_Differential,Sample_Weight,3287


In [13]:
# imputing soluble mediator data using QRILC
QRILC_imputation = function(dataset){
    wider_dataset = dataset %>%
        pivot_wider(names_from = Variable, values_from = Value)
    
    index_of_last_variable = length(colnames(wider_dataset))
    
    # normalizing data since that what the QRILC function wants
    # had to pseudo log transform to prevent Inf values
    QRILC_prep = wider_dataset[,10:all_of(index_of_last_variable)] %>%
         mutate_all(., function(x) log10(x + 1)) %>%
         as.matrix()

    imputed_QRILC_object = impute.QRILC(QRILC_prep, tune.sigma = 0.1)
    QRILC_log10_df = data.frame(imputed_QRILC_object[1]) 
    
    # converting back the original scale
    QRILC_df = QRILC_log10_df %>%
        mutate_all(., function(x) 10^x - 1)
                   
    imputed_dataset = data.frame(cbind(unique(dataset[,1:9]), QRILC_df)) %>%
        pivot_longer(cols = 10:all_of(index_of_last_variable), names_to = "Variable", values_to = "Value")
    
    return(imputed_dataset)
}

# calling fn
imputed_cell_differential_plasma_post_df = QRILC_imputation(cell_differential_plasma_post_df)
imputed_cell_differential_plasma_pre_df = QRILC_imputation(cell_differential_plasma_pre_df)
imputed_cell_differential_sputum_post_df = QRILC_imputation(cell_differential_sputum_post_df)
imputed_cell_differential_sputum_pre_df = QRILC_imputation(cell_differential_sputum_pre_df)
imputed_cytokine_plasma_post_df = QRILC_imputation(cytokine_plasma_post_df)
imputed_cytokine_plasma_pre_df = QRILC_imputation(cytokine_plasma_pre_df)
imputed_cytokine_sputum_post_df = QRILC_imputation(cytokine_sputum_post_df)
imputed_cytokine_sputum_pre_df = QRILC_imputation(cytokine_sputum_pre_df)
imputed_sterol_plasma_post_df = QRILC_imputation(sterol_plasma_post_df)
imputed_sterol_plasma_pre_df = QRILC_imputation(sterol_plasma_pre_df)
# sterol_sputum_post_df had no missing data
# sterol_sputum_pre_df had no missing data

Warning message in (function (x) :
“NaNs produced”
Warning message in (function (x) :
“NaNs produced”
Warning message in (function (x) :
“NaNs produced”
Warning message in (function (x) :
“NaNs produced”


In [14]:
# I think the data matrix doesn't like col names starting with numbers so it puts an X in front of it
# removing those X's prior to combining everything into 1 datafame
imputed_sterol_plasma_post_df = imputed_sterol_plasma_post_df %>%
     mutate(Variable = ifelse(grepl("X", Variable), 
                              unlist(strsplit(Variable, split = "X"))[2], Variable))
imputed_sterol_plasma_pre_df = imputed_sterol_plasma_pre_df %>%
    mutate(Variable = ifelse(grepl("X", Variable), 
                              unlist(strsplit(Variable, split = "X"))[2], Variable))

In [15]:
# combining all imputed data
# not sure why there are duplicates, but I removed duplicate entries
final_df = unique(rbind(imputed_cell_differential_plasma_post_df, imputed_cell_differential_plasma_pre_df, imputed_cell_differential_sputum_post_df,
                imputed_cell_differential_sputum_pre_df, imputed_cytokine_plasma_post_df, imputed_cytokine_plasma_pre_df, imputed_cytokine_sputum_post_df,
                imputed_cytokine_sputum_pre_df, lung_function_airways_post_df, lung_function_airways_pre_df, imputed_sterol_plasma_post_df, imputed_sterol_plasma_pre_df, 
                sterol_sputum_post_df, sterol_sputum_pre_df))
head(final_df)

Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Sample_Weight,817.00
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,TCC,395000.00
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,TCC_Weight,483.48
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Neutrophils_Percent,25.00
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Neutrophils,98750.00
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Neutrophils_Weight,121.00


In [16]:
# double checking that we were expecting 7,151 samples
dim(final_df)

[1] 7151   11

In [17]:
# exporting results
write.csv(final_df, paste0(Output,"/", cur_date, "_Imputed_Ozone_Data.csv"), row.names = FALSE)